In [ ]:
!pip install -q git+https://github.com/tensorflow/docs

import matplotlib.pyplot as plt
import numpy as np
import pandas as pd

try:
    %tensorflow_version 2.x
except Exception:
    pass

import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers

import tensorflow_docs as tfdocs
import tensorflow_docs.plots
import tensorflow_docs.modeling


In [ ]:
!wget https://cdn.freecodecamp.org/project-data/health-costs/insurance.csv
dataset = pd.read_csv('insurance.csv')
dataset.tail()


In [ ]:
# Convert categorical columns to numeric
dataset = pd.get_dummies(dataset, columns=['sex','smoker','region'], drop_first=True)
dataset.head()
from sklearn.model_selection import train_test_split

train_dataset, test_dataset = train_test_split(dataset, test_size=0.2, random_state=42)
train_labels = train_dataset.pop('expenses')
test_labels = test_dataset.pop('expenses')
from sklearn.preprocessing import StandardScaler

scaler = StandardScaler()
train_dataset = scaler.fit_transform(train_dataset)
test_dataset = scaler.transform(test_dataset)
model = tf.keras.Sequential([
    layers.Dense(64, activation='relu', input_shape=[train_dataset.shape[1]]),
    layers.Dense(64, activation='relu'),
    layers.Dense(1)  # output layer for regression
])

model.compile(optimizer='adam', loss='mse', metrics=['mae', 'mse'])
history = model.fit(
    train_dataset, train_labels,
    epochs=200,
    validation_split=0.2,
    verbose=0  # change to 1 to see training progress
)
# Evaluate
loss, mae, mse = model.evaluate(test_dataset, test_labels, verbose=2)
print("Testing set Mean Abs Error: {:5.2f} expenses".format(mae))

if mae < 3500:
    print("You passed the challenge. Great job!")
else:
    print("The Mean Abs Error must be less than 3500. Keep trying.")

# Plot predictions
test_predictions = model.predict(test_dataset).flatten()

a = plt.axes(aspect='equal')
plt.scatter(test_labels, test_predictions)
plt.xlabel('True values (expenses)')
plt.ylabel('Predictions (expenses)')
lims = [0, 50000]
plt.xlim(lims)
plt.ylim(lims)
_ = plt.plot(lims, lims)
